In [7]:
import pickle
import pandas as pd

In [26]:
with open('./data/transformed_data/teams_stats.pickle', 'rb') as team_stats:
    teams_stats = pickle.load(team_stats)

print(teams_stats)

<_io.BufferedReader name='./data/transformed_data/teams_stats.pickle'>


In [ ]:
with open('./data/transformed_data/players_stats.pickle', 'rb') as play_stats:
    players_stats = pickle.load(play_stats)

print(players_stats)

In [ ]:
with open('./data/transformed_data/players_advanced_stats.pickle', 'rb') as play_advanced_stats:
    players_advanced_stats = pickle.load(play_advanced_stats)

print(players_advanced_stats)

In [11]:
game_types = ["playoffs", "leagues"]
#years = list(range(2001, 2023))
years = list(range(2001, 2002))
stats = {}
df_columns = ["ID", "age", "games_played", "games_started", "avg_minutes_played", "WS48", "team_successes", "defensive", "most_improved", "most_valuable", "rookie", "sixth_man"]
df_template = pd.DataFrame(columns=df_columns)

In [12]:
for year in years:
    stats[year] = df_template

In [17]:
# GAMES PLAYED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    stats[year]['ID'] = [player for player in player_list]
    stats[year] = stats[year].set_index("ID")
    for player_id in player_list:
        games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    games_played += players_stats[year][game_type].query(f"ID == '{player_id}'")["G"].iloc[0]
                except:
                    games_played += 0
                
            stats[year].loc[[f'{player_id}'], ['games_played']] = games_played

In [18]:
# GAMES STARTED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        games_started = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    games_started += players_stats[year][game_type].query(f"ID == '{player_id}'")["GS"].iloc[0]
                except:
                    games_started += 0
            
            stats[year].loc[[f'{player_id}'], ['games_started']] = games_started

In [19]:
# AVERAGE MINUTES PLAYED FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        avg_minutes_played = 0
        avg_minutes_played_x_games = 0
        total_games = 0
        minutes_played_per_game = 0
        games_played = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    minutes_played_per_game = players_stats[year][game_type].query(f"ID == '{player_id}'")["MP"].iloc[0]
                    games_played = players_stats[year][game_type].query(f"ID == '{player_id}'")["G"].iloc[0]
                except:
                    minutes_played_per_game += 0
                    games_played += 0
                
            avg_minutes_played_x_games += minutes_played_per_game * games_played
            total_games += games_played
            
        if total_games != 0:
            avg_minutes_played = round(avg_minutes_played_x_games/total_games, 2)
        else:
            avg_minutes_played = 0
        
        stats[year].loc[[f'{player_id}'], ['avg_minutes_played']] = avg_minutes_played

In [22]:
# WIN SHARES PER 48 MINUTES FOR EACH PLAYER
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    for player_id in player_list:
        minutes_played = 0
        win_shares = 0
        win_share_48 = 0
        for game_type in game_types:
            if player_id in player_list:
                try:
                    minutes_played = players_stats[year][game_type].query(f"ID == '{player_id}'")["MP"].iloc[0]
                    win_shares = players_advanced_stats[year][game_type].query(f"ID == '{player_id}'")["WS"].iloc[0]
                except:
                    minutes_played += 0
                    win_shares += 0
            
        if minutes_played != 0:
            win_share_48 = round((win_shares/minutes_played) * 48, 2)
        else:
            win_share_48 = 0
        
        stats[year].loc[[f'{player_id}'], ['WS48']] = win_share_48

In [23]:
print(stats[2001])

           age games_played games_started avg_minutes_played   WS48   
ID                                                                    
smithmi02  NaN           79            29               20.4   8.47  \
bryanko01  NaN           84            84              41.38  13.26   
mutomdi01  NaN           98            98              36.42  11.27   
carrch01   NaN           35             0                8.8   4.91   
averywi01  NaN           57             0               8.32    0.0   
...        ...          ...           ...                ...    ...   
shawbr01   NaN           96            28               22.1   5.24   
portech01  NaN           51            35               22.5   0.43   
barryjo01  NaN           69             2              15.45   7.07   
johnsav01  NaN           68            20              23.14   5.31   
milesda01  NaN           81            21               26.3   5.48   

          team_successes defensive most_improved most_valuable rookie   
ID 